 # Benchmarking

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from benchmark import benchmark_csv
from benchmark.process import runnable, run
from os import PathLike, chdir
from pathlib import Path
from benchmark.api import Api, ApiBefore, ApiAfter
from time import sleep


if not "PROJECT_DIR" in globals():
    PROJECT_DIR = Path().resolve().parent
    chdir(PROJECT_DIR)


def out(name: str) -> PathLike:
    return PROJECT_DIR / "out" / f"{name}.csv"

## Clean up

Making sure nothing is running before the benchmarks.

In [ ]:
run("make remove-before")
run("make remove-after")
run("make remove-queue")

## Compose Benchmarks

In [4]:
states = ["before", "after", "queue"]

### Cold Start

In [10]:
for state in states:
    benchmark_csv(
        out(f"cold-start_{state}"),
        func=runnable(f"make start-{state}"),
        after=runnable(f"make remove-{state}"),
    )

### Warm Start

In [ ]:
for state in states:
    run(f"make start-{state} && make stop-{state}")
    benchmark_csv(
        out(f"warm-start_{state}"),
        func=runnable(f"make start-{state}"),
        after=runnable(f"make stop-{state}"),
    )
    run(f"make remove-{state}")

### Stop

In [12]:
for state in states:
    benchmark_csv(
        out(f"stop_{state}"),
        before=runnable(f"make start-{state}"),
        func=runnable(f"make stop-{state}"),
    )
    run(f"make remove-{state}")

### Remove

In [13]:
for state in states:
    benchmark_csv(
        out(f"remove_{state}"),
        before=runnable(f"make start-{state}"),
        func=runnable(f"make remove-{state}"),
    )

## API

In [7]:
apis: dict[str, Api] = {"before": ApiBefore(), "after": ApiAfter()}

In [21]:
for state, api in apis.items():
    run(f"make start-{state}")
    sleep(1)
    api.create_user("albert", "einstein", "albert.einstein@example.com")
    api.login("albert", "einstein")

    benchmark_csv(
        out(f"create-database_{state}"),
        func=lambda index: api.create_database(f"Database {index+1}"),
    )

    if state == "before":
        run("make remove-before-userdb")

    database_id = api.create_database("Test Database")

    benchmark_csv(
        out(f"create-table_{state}"),
        func=lambda index: api.create_table(database_id, f"Table {index+1}"),
    )

    table_id = api.create_table(database_id, "Test Table")

    benchmark_csv(
        out(f"insert-row_{state}"),
        func=lambda index: api.insert_row(database_id, table_id, {"id": index + 1}),
    )

    benchmark_csv(
        out(f"update-row_{state}"),
        func=lambda index: api.update_row(
            database_id, table_id, {"id": index + 1}, {"id": index + 11}
        ),
    )

    benchmark_csv(
        out(f"delete-row_{state}"),
        func=lambda index: api.delete_row(database_id, table_id, {"id": index + 11}),
    )

    benchmark_csv(
        out(f"import-csv_{state}"),
        func=lambda _: api.import_csv(database_id, table_id, PROJECT_DIR / "test.csv"),
    )

    benchmark_csv(
        out(f"create-query_{state}"),
        func=lambda index: api.create_query(
            database_id, f"SELECT id FROM test_table WHERE id = {index+101}"
        ),
    )

    query_result = api.create_query(database_id, "SELECT id FROM test_table")

    benchmark_csv(
        out(f"execute-query_{state}"),
        func=lambda _: api.execute_query(database_id, query_result["id"]),
    )

    run(f"make remove-{state}")